# Image Classification Pipeline

This notebook walks through building a **complete deep learning pipeline** for image classification:
1. Data loading & preprocessing  
2. Model definition  
3. Training & evaluation  
4. Predictions on new images  

We’ll use **PyTorch** for this implementation.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

## 2. Data Preparation
We use `torchvision.datasets` to load image data.  
For demonstration, we’ll use **CIFAR-10** (10-class dataset with 60,000 images).

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_data = datasets.CIFAR10(root="./data", train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10(root="./data", train=False,
                             download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

class_names = train_data.classes
print("Classes:", class_names)

## 3. Define Model
We’ll use a **pretrained ResNet18** and fine-tune it for CIFAR-10.

In [ ]:
model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## 4. Training Setup

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

## 5. Training Loop

In [ ]:
def train(model, loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(loader):.4f}")

train(model, train_loader, criterion, optimizer, epochs=5)

## 6. Evaluation

In [ ]:
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy: {100 * correct / total:.2f}%")

evaluate(model, test_loader)

## 7. Inference on a New Image

In [ ]:
from PIL import Image

def predict_image(path, model):
    model.eval()
    image = Image.open(path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    return class_names[predicted.item()]

# Example usage (provide your own image path)
# print(predict_image("sample_image.jpg", model))

## Summary
- We built a **full pipeline**: preprocessing → training → evaluation → prediction.  
- Using **transfer learning (ResNet18)** speeds up training and improves accuracy.  
- This approach works for CIFAR-10 but can easily adapt to **custom datasets** by changing `datasets.ImageFolder`.